In [ ]:
%load_ext autoreload
%autoreload 1

%aimport Seasonal_Outliers
import importlib

importlib.reload(Seasonal_Outliers)

import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable

import Ipynb_importer
from numba import jit,cuda

from prophet import Prophet
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

def read(file):
    df = pd.read_csv(file)
    print(df.head())
    
    
    df['started_datetime'] = pd.to_datetime(df['started_datetime'], errors='coerce')
    df['started_date'] = df['started_datetime'].dt.date
    
    df['ended_datetime'] = pd.to_datetime(df['ended_datetime'], errors='coerce')
    df['ended_date'] = df['ended_datetime'].dt.date

    
    df['Day of week'] = df['started_datetime'].dt.dayofweek
    dt_wkends = df[df['Day of week']>4]['started_date'].unique()
    
    return df,dt_wkends.tolist()


def combine_df(idir,files):
    df = pd.DataFrame()
    dt_non_work = []#non-working day
    for fi in files:
        dfi,dt_wkends = read(idir+fi)
        df = df.append(dfi,ignore_index=True)
        dt_non_work+=dt_wkends
    holiday = pd.to_datetime(pd.Series(['2021-07-04','2021-07-05','2021-09-06','2021-09-16','2021-10-11'])).dt.date.unique()
    dt_non_work = np.append(dt_non_work,holiday)#Labor Day

    return df,dt_non_work


def vis(idir,outliers,col,title):
    # print(outliers)
    dt_range = pd.to_datetime(pd.date_range(start='07/01/2021', end='10/31/2021')).date
    df = pd.DataFrame(columns=outliers[col[0]].unique(),index=dt_range)
    df = df.fillna(0).astype('float')

    for idx in df.index:#datetime
        vals = outliers[outliers[col[1]]==idx][[col[0]]].values
        # print(vals)
        for val in vals:
            v = str(val[0])
            df.loc[idx,v] = outliers.loc[(outliers[col[1]]==idx)&(outliers[col[0]].astype(str)==v)]['norm_resid'].values[0]

    figsize=(30, 12)
    fig, ax = plt.subplots(figsize=figsize)
    # my_colors=['whitesmoke','orangered']
    
    ax = sns.heatmap(df,cmap='bwr',center=0,zorder=1, yticklabels = df.index,cbar_kws={"pad": 0.05})#square=True,
    

    for i in range(df.shape[1]+1):
        ax.axvline(i, color='white', lw=0.4, zorder=2)
        
        
    x1 = 0
    x2 = len(df.columns)

    ylabels = ax.get_yticklabels()
    ys = [ y.get_position()[1] for y in ylabels if str(y.get_text()) in ["2021-09-01","2021-09-02"]]
    y1 = ys[0]-0.5
    y2 = ys[1]+0.5
    print(ys)
    # ax.fill_between([x1,x2],ida_window[0],ida_window[1], color='lightskyblue', alpha=0.4)
    ax.fill_between([x1,x2],y1,y2, alpha=0.5, color='none',edgecolor='blue', zorder=3,label='Ida')
    
    ys2 = [ y.get_position()[1] for y in ylabels if str(y.get_text()) in ["2021-08-21","2021-08-23"]]
    y12 = ys2[0]-0.5
    y22 = ys2[1]+0.5
    print(ys2)
    ax.fill_between([x1,x2],y12,y22, alpha=0.8, color='none',edgecolor='gold', zorder=3,label='Henri')
    
    ys3 = [ y.get_position()[1] for y in ylabels if str(y.get_text()) in ["2021-07-08","2021-07-09"]]
    y13 = ys3[0]-0.5
    y23 = ys3[1]+0.5
    print(ys3)
    ax.fill_between([x1,x2],y13,y23, alpha=0.5, color='none',edgecolor='green', zorder=3,label='Elsa')
    
    ys4 = [ y.get_position()[1] for y in ylabels if str(y.get_text()) in ['2021-10-25','2021-10-26']]
    y14 = ys4[0]-0.5
    y24 = ys4[1]+0.5
    print(ys4)
    ax.fill_between([x1,x2],y14,y24, alpha=0.5, color='none',edgecolor='deeppink', zorder=3,label="Nor'eastern")
    

    

    for i, label in enumerate(ylabels):
        if i % 2 == 0:
            label.set_visible(True)
        else:
            label.set_visible(False)
            
   

        
    # ax.set_title(title)
    ax.legend(ncol=4,loc='upper right',fontsize=22)

    fig = plt.gcf()
    cax = fig.axes[-1]
    plt.subplots_adjust(left=0.15, right=0.95, bottom=0.15, top=0.9, wspace=0.2, hspace=0.2)
    cax.set_position([.796, .2, .03, .6]) 
    plt.savefig(idir+'result/'+title.replace(':','_').replace('/',' ')+'.png',format='png',bbox_inches='tight',dpi=300)
    plt.show()

    

@jit(target_backend='cuda')     
def find_outliers(idir,df,title):
    if "starts" in title:
        col = ["start_station_name",'started_date']
    elif "ends" in title:
        col = ["end_station_name",'ended_date']
        
    gps = df.groupby([col[0]])
    outs = pd.DataFrame()
    stas = pd.DataFrame()
    for nm,df2 in gps:
        # print(df2)
        nm = str(nm)
        # print(nm)
        df2['index'] = df2.index
        sta_df = pd.DataFrame()
        sta_df['cnt'] = df2.groupby([col[1]]).count()['index']
        key='cnt'
        if sta_df['cnt'].mean()<36:
            continue
        out = pd.DataFrame()
        tmp,forecast = Seasonal_Outliers.seasonal_de_day(sta_df,key,plot=False)
        if len(tmp)>0:
            out[col[1]] = tmp
            out[col[0]] = nm
            outs = pd.concat([outs,out[col]],ignore_index=True)
            forecast[col] = nm
            stas = pd.concat([stas,forecast],ignore_index=True)
    
    sst = stas[['ds','norm_resid',col[0]]]
    print(sst,outs)
    sst['ds'] = pd.to_datetime(sst['ds'], errors='coerce')
    sst[col[1]] = sst['ds'].dt.date
    outs = pd.merge(outs,sst,on=col)
    print(outs)
    
    vis(idir,outs,col, title)
    stas.to_csv(idir+'result/'+title.replace(':','_').replace('/',' ')+'.csv',index=False)


    
    
def run(idir,files,_title):
    
    # df,dt_non_work = combine_df(idir,files)

    keys_dict = {
        "stations' starts":[''],
        "stations' ends":[''],
                 
                }

    for k,v in keys_dict.items():
        if "starts" in k:
            title = _title + "stations' starts"
        if "ends" in k:
            title = _title + "stations' ends"
       
        find_outliers(idir,df,title)
        
 


    
    


In [ ]:
input_dir = ""
files = [i for i in os.listdir(input_dir) if os.path.splitext(i)[1] == '.csv']
    
_title = 'Daily Stationarity of '
df,dt_non_work = combine_df(input_dir,files)  


In [ ]:
%load_ext autoreload
%autoreload 1
%aimport Seasonal_Outliers
import importlib
importlib.reload(Seasonal_Outliers)
run(input_dir, files, _title)